In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
import time
from tqdm import trange, tqdm
import pandas as pd

In [2]:
def is_present_xpath(text):    
    try:
        driver.find_element_by_xpath("//*[text()='" + text + "']")
    except NoSuchElementException:
        return False
    
    return True


def is_present_id(text):    
    try:
        driver.find_element_by_id(text)
    except NoSuchElementException:
        return False
    
    return True


def is_present_class_name(text):    
    try:
        driver.find_element_by_class_name(text)
    except NoSuchElementException:
        return False
    
    return True

In [6]:
# not in use
def check_if_loading():
    t0= time.clock()
    while is_present_xpath('End-to-end encrypted'):
        t1 = time.clock() - t0
        if t1 > 10:
            driver.refresh()
        else:
            continue

In [7]:
df = pd.read_excel("./contacts.ods", engine="odf")
phone = df['phone']
phone_country = '972'
family_name = df['last name']
first_name = df['first name']
link = 'http://yc.ms/gdi/KInvcU'
message = 'מתרגשים להזמין אתכם לחתונה שלנו שתתקיים במתחם פתוח ובכפוף לתו הירוק.'

In [63]:
driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver')
driver.get("https://web.whatsapp.com/")
driver.maximize_window()

WebDriverWait(driver, 600).until(lambda driver: driver.find_element_by_xpath("//*[text()='To use WhatsApp on your computer:']"))
while is_present_xpath("To use WhatsApp on your computer:"):
    if is_present_xpath("Click to reload QR code"):
        qr = WebDriverWait(driver, 10).until(lambda driver: driver.find_element_by_xpath("//*[text()='Click to reload QR code']"))
        qr.click()
    if is_present_xpath("Keep your phone connected"):
        break
    else:
        continue
driver.get("https://api.whatsapp.com/send?phone=" + phone_country + phone[0] + "&lang=he")

In [57]:
sent = pd.DataFrame(columns = ["first name", "last name", "phone"])
invalid_phone = pd.DataFrame(columns = ["first name", "last name", "phone"])
for i in tqdm(range(len(df))):
    bad = False
    driver.get("https://api.whatsapp.com/send?phone=" + phone_country + phone[i] + "&lang=he")
    wait = WebDriverWait(driver, 600).until(lambda driver: driver.find_element_by_id("action-button"))
    wait.click()
    time.sleep(1)
    wait2 = WebDriverWait(driver, 600).until(lambda driver: driver.find_element_by_xpath("//*[text()='השתמשו ב-WhatsApp Web']"))
    wait2.click()
    while not is_present_class_name("_1UWac._1LbR4"):
        if is_present_xpath("Phone number shared via url is invalid."):
            sent = sent.append(df.iloc[i], ignore_index=True)
            WebDriverWait(driver, 600).until(lambda driver: driver.find_element_by_xpath("//*[text()='OK']")).click()
            bad = True
            break
        else:
            continue
    if bad:
        print('Caught invalid number')
        continue
    else:
        wait3 = WebDriverWait(driver, 600).until(lambda driver: driver.find_element_by_class_name("_1UWac._1LbR4"))
        while driver.find_element_by_xpath("//*[text()='Type a message']").is_displayed():
            wait3.send_keys(Keys.CONTROL + "A")
            wait3.send_keys(link)
            time.sleep(2)
        while not is_present_class_name('a-HbF'):
            wait3.send_keys(Keys.CONTROL + "A")
            wait3.send_keys(link)
            time.sleep(2)
        wait4 = WebDriverWait(driver, 600).until(lambda driver: driver.find_element_by_class_name("a-HbF"))
        wait4.click()
        send = driver.find_element_by_class_name("_4sWnG").click()
        while driver.find_element_by_xpath("//*[text()='Type a message']").is_displayed():
            wait3.send_keys(Keys.CONTROL + "A")
            wait3.send_keys(message)
        send2 = driver.find_element_by_class_name("_4sWnG").click()
        time.sleep(2)
        sent = sent.append(df.iloc[i], ignore_index=True)

100%|██████████| 25/25 [06:21<00:00, 15.25s/it]
